# 一、神经网络基本原理
## 1.1 什么是神经网络？
神经网络启发于生物学的神经元连接，通过大量的模型参数学习数据中的有效特征来完成对应的任务。核心过程包括前向传播和后向传播，前向传播通过设计各种模型结构来更有效的特征提取，后向传播根据损失函数完成能够降低任务损失的参数更新。
## 1.2 反向传播算法（Backpropagation）
反向传播是基于链式法则的参数梯度计算方法。前向传播阶段会构建计算图，记录各层的操作与中间结果；输出通过损失函数聚合为一个标量 loss。随后，反向传播沿计算图自后向前递推loss对每层参数的梯度。最终，这些梯度通过梯度下降等优化算法用于更新模型参数。
- 链式法则是用于求复合函数导数的公式，表示复合函数的导数等于各个内部函数导数的乘积。在深度学习中，它是反向传播算法的数学基础，用来将损失函数对输出的梯度，通过层与层之间的依赖关系，一层层地传回去，最终得到每个参数的梯度，从而进行梯度下降优化。
- 为什么是最后的loss是标量？
    - 标量才可以有唯一方向降低损失：
        
        假设神经网络最终输出是：
        - $\hat{y} = f(x; \theta)$
        - 损失函数：$L = \text{Loss}(\hat{y}, y)$

        需要计算：

        $$
        \frac{\partial L}{\partial \theta}
        $$

        这只能在 $L$ 是标量的情况下才能得到明确的导数，保证有唯一方向去更新参数。如果 $L$ 是向量，比如 shape = (batch\_size,)，将无法直接用链式法则去求导。

- 我理解的误区：每个样本的loss组成的向量并不是进行反向传播的损失L，我们能够直接根据损失函数求每个样本的导数，但其导数也是对于聚合后的、反向传播的损失L
    
    假如定义的 loss 是：

    $$
    L = \sum_{i=1}^n \frac{1}{2}(y_i - \hat{y}_i)^2
    $$

    我们想要求：

    $$
    \frac{\partial L}{\partial \hat{y}_i}
    $$


    - 方法一：人类理解的

        对每一项求导：

        $$
        \frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i
        $$

        完全正确，没问题！


    - 方法二：自动求导框架的实际“链式传播路径”

        框架是这样理解你的写法的：

        1. **先计算每个样本的损失：**

        $$
        L_i = \frac{1}{2}(y_i - \hat{y}_i)^2
        $$

        2. **再通过聚合：**

        * 使用 `sum`：

            $$
            L = \sum_i L_i
            $$

            对 $L_i$ 的导数是 $\frac{\partial L}{\partial L_i} = 1$

        3. **再通过链式法则：**

        $$
        \frac{\partial L}{\partial \hat{y}_i} = \frac{\partial L}{\partial L_i} \cdot \frac{\partial L_i}{\partial \hat{y}_i} = 1 \cdot (\hat{y}_i - y_i)
        $$

        所以结果依然是：
        >
        > $$
        > \boxed{\frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i}
        > $$
        >
        > 和你直觉上的结果完全一样。


## 1.3 梯度下降再认识
### 1️⃣ 梯度的定义

对标量 loss $L$ 关于参数张量 $W$ 求导：

$$
\nabla_W L = \frac{\partial L}{\partial W}
$$

* 结果是与参数同形状的张量，每个元素对应一个参数的偏导数
* 整个梯度张量在高维参数空间中构成了 loss 上升最快的方向，负梯度方向就是下降最快的方向
* 每个元素 $\frac{\partial L}{\partial W_{ij}}$ 是整个梯度向量的一个分量，loss这个分量下增加的瞬时变化率（即如果只改变该参数一个单位，loss 的变化量）

**一句话总结：** 在深度学习中，loss 是参数张量的函数，其梯度是一个向量，表示当前参数点处 loss 上升最快的方向。每个参数的偏导数（即梯度分量）表示该参数对 loss 的局部影响程度，整体梯度方向指引我们如何更新参数以最快地减小 loss。

**形象比喻：**

想象三维空间的地形（loss landscape）：

* 你站在某一点 $W$ 

* 沿 x、y、z 方向，你能测到坡度（梯度分量）

* 把三个方向的坡度合成一个箭头（梯度向量）

* 顺着负梯度箭头走，就能最快下山（loss下降）


---

### 2️⃣ 梯度下降更新

梯度下降是利用梯度信息更新参数，使 loss 下降最快的方法。具体步骤如下：

1. **计算梯度**
   对当前参数 $W$ 计算 loss 的梯度：

   $$
   \nabla_W L = \frac{\partial L}{\partial W}
   $$

2. **沿负梯度方向更新参数**
   使用学习率 $\eta$ 控制步长，更新公式为：

   $$
   W \gets W - \eta \nabla_W L
   $$

   * 这里 $-\nabla_W L$ 是 loss 下降最快的方向。
   * 学习率 $\eta$ 决定每一步走多远，过大可能发散，过小收敛慢。

3. **重复迭代**
   在训练过程中，多次计算梯度并更新参数，逐步减小 loss，直到满足停止条件（如 loss 足够小或达到最大迭代次数）。

---

**形象比喻：**

* 想象你站在三维地形的某一点：

  * 梯度告诉你“地面坡度有多陡、哪个方向坡最大”。
  * 负梯度方向就是下山最快的路。
  * 每次沿这个方向走一步，就相当于更新参数 $W$ 一次。
* 反复多次，就像沿山坡下滑，最终到达低谷（loss 最小）。


